In [1]:
#import OpenCV module
import cv2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from deepface import DeepFace
import firebase_admin
from firebase_admin import firestore_async, credentials, messaging
from datetime import datetime
from google.cloud import storage

predicted = []
default_app = firebase_admin.initialize_app()

#function to get subject names
def get_subject():
    file = open("/Users/wongtiksoon/Desktop/SMS/subject.txt", 'r+')
    data = file.readlines()
    datas = []
    for name in data:
        name = name.replace('\n', '')
        datas.append(name)
    datas.insert(0, "")
    return datas

#function to detect face
def detect_face (img):
    #convert the test image to gray image
    gray = cv2.cvtColor (img, cv2.COLOR_BGR2GRAY)
    #load OpenCV face detector
    face_cas = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cas.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    #if no faces are detected then return image
    if (len (faces) == 0):
        return None, None
    #extract the face
    x = faces[0][0]
    y = faces[0][1]
    w = faces[0][2]
    h = faces[0][3]
    #return only the face part
    return gray[y: y+w, x: x+h], faces[0]

#this function will read all persons' training images, detect face #from each image
#and will return two lists of exactly same size, one list
def prepare_training_data(data_folder_path):
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    for dir_name in dirs:
        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
        #------STEP-2--------
        #extract label number of subject from dir_name
        #format of dir name = slabel
        #, so removing letter 's' from dir_name will give us label
        label = int(dir_name.replace("s", ""))
        #build path of directory containing images for current subject subject
        #sample subject_dir_path = "training-data/s1"
        subject_dir_path = data_folder_path + "/" + dir_name
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        #------STEP-3--------
        #go through each image name, read image,
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue;
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name
            #read image
            image = cv2.imread(image_path)
            #display an image window to show the image
#             cv2.imshow("Training on image...", image)
#             cv2.waitKey(100)
            #detect face
            face, rect = detect_face(image)
            #------STEP-4--------
            #we will ignore faces that are not detected
            if face is not None:
                #add face to list of faces
                faces.append(face)
                #add label for this face
                labels.append(label)
                cv2.destroyAllWindows()
                cv2.waitKey(1)
                cv2.destroyAllWindows()
    return faces, labels

#function to draw rectangle on image
#according to given (x, y) coordinates and
#given width and heigh
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

#function to draw text on give image starting from
#passed (x, y) coordinates.
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

#this function recognizes the person in image passed
#and draws a rectangle around detected face with name of the subject
def predict(test_img):
    global label_text
    global emotion
    #make a copy of the image as we don't want to change original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)
    if face is not None:
        #predict the image using our face recognizer
        label = face_recognizer.predict(face)
        #get name of respective label returned by face recognizer
        subjects = get_subject()
        label_text = subjects[label[0]]
        predicted.append(label_text)
        #detect emotion of face
        result = DeepFace.analyze(test_img, enforce_detection=False)
        emotion = result[0]["dominant_emotion"]
        #draw a rectangle around face detected
        draw_rectangle(img, rect)
        #draw name of predicted person
        draw_text(img, label_text + "_" + emotion, rect[0], rect[1]-5)
        print("Confidence score being " + label_text + " is " + str(label[1]))
        return img
    else:
        print("Face not found!")
        return None
        
    
    
def upload(name, emotion, image, img_name):
    
    #Upload the attendance with name, date, time and emotion.
    
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/Users/wongtiksoon/.config/gcloud/application_default_credentials.json'
    db = firestore_async.Client(project="studentmonitoringsystem-2c071")

    # datetime object containing current date and time
    upload = datetime.now()

    # dd/mm/YY H:M:S
    date = now.strftime("%d-%m-%Y")
    time = now.strftime("%H:%M:%S")
    
    attendance = db.collection("attendance").where("studentid", "!=", "")
    counter = 0

    for doc in attendance.stream():   
        counter = counter + 1
        
    studentdata = db.collection("student").where("studentid", "!=", "")
    students = []
    for student in studentdata.stream():   
        formattedData = student.to_dict()
        students.append(formattedData)

    for student in students:
        if student["name"] == name:
            studentid = student["studentid"]

    attendanceid = "ATT" + "{:06d}".format(counter+1)
    
    #upload image
    client = storage.Client()
    bucket_name = 'studentmonitoringsystem-2c071.appspot.com'
    bucket = client.get_bucket(bucket_name)
    # posting to firebase storage
    imageBlob = bucket.blob("/")
    # imagePath = [os.path.join(self.path,f) for f in os.listdir(self.path)]
    imagePath = image
    imageBlob = bucket.blob(img_name)
    imageBlob.upload_from_filename(imagePath)
    
    firebase_storageURL = 'https://firebasestorage.googleapis.com/v0/b/{}/o/{}?alt=media'.format(bucket_name, img_name)
    
    db.collection("attendance").document(attendanceid).set({
        "attendanceid": attendanceid,
        "studentid": studentid,
        "emotion": emotion,
        "date": date,
        "time": time,
        "image": firebase_storageURL
    })
    
def sendPush(name, title, msg, dataObject=None):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/Users/wongtiksoon/.config/gcloud/application_default_credentials.json'
    db = firestore_async.Client(project="studentmonitoringsystem-2c071")
    
    studentdata = db.collection("student").where("studentid", "!=", "")
    parents = []
    students = []
    tokens = []
    for student in studentdata.stream():   
        formattedData = student.to_dict()
        students.append(formattedData)

    for student in students:
        if student["name"] == name:
            parent = student["parentid"]

    parentdata = db.collection("users").where("id", "==", parent)
    for parent in parentdata.stream():   
        formattedData = parent.to_dict()
        parents.append(formattedData)
        
    token = parents[0]["token"]
    tokens.append(token)

    message = messaging.MulticastMessage(
        notification=messaging.Notification(
            title=title,
            body=msg
        ),
        data=dataObject,
        tokens=tokens,
    )

    # Send a message to the device corresponding to the provided
    # registration token.
    response = messaging.send_multicast(message)
    # Response is a message ID string.
    print('Successfully sent message:', response)

In [2]:
#prepare training data
#data will be in two lists of same size
#one list will contain all the faces
#other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("/Users/wongtiksoon/Desktop/SMS/training-data")
print("Data prepared")
#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))
#create our LBPH face recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
#train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))

Preparing data...
Data prepared
Total faces:  57
Total labels:  57


In [3]:
data_folder_path = "/Users/wongtiksoon/Desktop/SMS/test-data"
dirs = os.listdir(data_folder_path)

predicted = []
for img in dirs:
    if img.startswith("."):
            continue;
    image_path = data_folder_path+ "/" + img
    #load test images
    test_img = cv2.imread(image_path)
    #perform a prediction
    predicted_img = predict(test_img)
    cv2.imshow(img, predicted_img)


print("Prediction complete")
print("Total predicted: ", len(predicted))
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.destroyAllWindows()

Action: race: 100%|███████████████████████████████| 4/4 [00:03<00:00,  1.32it/s]


Confidence score being Mac is 56.33046790946825


Action: race: 100%|███████████████████████████████| 4/4 [00:01<00:00,  3.54it/s]


Confidence score being Wong is 57.203247756198735


Action: race: 100%|███████████████████████████████| 4/4 [00:01<00:00,  3.66it/s]


...

Confidence score being Wong is 68.45484777732013


Action: race: 100%|███████████████████████████████| 4/4 [00:01<00:00,  3.77it/s]


Confidence score being Daniel is 68.75643043118845


Action: race: 100%|███████████████████████████████| 4/4 [00:01<00:00,  3.72it/s]


Confidence score being Justin is 58.65024307354299
Prediction complete
Total predicted:  15


In [4]:
import os
path = '/Users/wongtiksoon/Desktop/SMS/test-data'
files = os.listdir(path)

names = []
for name in files:
    if name.startswith("."):
        continue;
    name = name.replace('.jpeg', "")
    name = ''.join(i for i in name if not i.isdigit())
    names.append(name)
print(names)
print(predicted)

['Mac', 'Tee', 'Adam', 'Daniel', 'Mac', 'Christine', 'Lim', 'Justin', 'Wong', 'Adam', 'Jacob', 'Adam', 'Wong', 'Sam', 'Justin']
['Mac', 'Wong', 'Adam', 'Daniel', 'Mac', 'Christine', 'Lim', 'Justin', 'Wong', 'Wong', 'Adam', 'Adam', 'Wong', 'Daniel', 'Justin']


In [5]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(names, predicted)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 73.33%


In [ ]:
image_path = "/Users/wongtiksoon/Desktop/SMS/test-data/Wong12.jpeg"
uploaded_image = cv2.imread(image_path)
predicted_img = predict(uploaded_image)

now = datetime.now()
dt = now.strftime("%d_%m_%Y_%H_%M_%S")

path = "/Users/wongtiksoon/Desktop/SMS/Attendance"

if predicted_img is not None:
    cv2.imshow("Attendance", predicted_img)

    # the format for storing the images scrreenshotted
    img_name = f'attendance_{dt}.jpeg'
            # saves the image as a jpeg file
    new_path = os.path.join(path , img_name)
    cv2.imwrite(new_path, predicted_img)
            
    upload(label_text, emotion, new_path, img_name)
    sendPush(label_text, "SMS", "Attendance taken by student.")
    print("Attendance uploaded!")
else:
    print("Attendance failed to capture!")

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.destroyAllWindows()

In [6]:
# intialize the webcam and pass a constant which is 0
cam = cv2.VideoCapture(0)

# while loop
while True:
    # intializing the frame, ret
    ret, frame = cam.read()
    # if statement
    if not ret:
        print('failed to grab frame')
        break
    # the frame will show with the title of test
    cv2.imshow('Attendance', frame)
    #to get continuous live video feed from my laptops webcam
    k  = cv2.waitKey(1)
    # if the escape key is been pressed, the app will stop
    if k%256 == 27:
        print('escape hit, closing the camera')
        break
    # if the spacebar key is been pressed
    # screenshots will be taken
    elif k%256 == 32:
        # datetime object containing current date and time
        now = datetime.now()
        dt = now.strftime("%d_%m_%Y_%H_%M_%S")
        
        predicted_img = predict(frame)
        
        if predicted_img is not None:
            cv2.imshow("Attendance", predicted_img)
            # the format for storing the images screenshotted
            img_name = f'attendance_{dt}.jpeg'
            path = "/Users/wongtiksoon/Desktop/SMS/Attendance"
            new_path = os.path.join(path , img_name)
            cv2.imwrite(new_path, predicted_img)
            
            upload(label_text, emotion, new_path, img_name)
            sendPush(label_text, "SMS", "Attendance taken by student.")
            print("Attendance uploaded!")
            break
        else:
            print("Attendance failed to capture!")
            break

# release the camera
cam.release()

# stops the camera window
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.destroyAllWindows()

Action: race: 100%|███████████████████████████████| 4/4 [00:01<00:00,  3.50it/s]


Confidence score being Wong is 52.17527263667704
Attendance uploaded!
